## Datasets

### Trespass Towing Report
https://data.montgomerycountymd.gov/Consumer-Housing/Trespass-Towing-Report/i6vn-3s6e/about_data

### Places
https://chronicdata.cdc.gov/500-Cities-Places/PLACES-Census-Tract-Data-GIS-Friendly-Format-2024-/yjkw-uj5s/about_data

### EPA Smart Locations
https://www.epa.gov/system/files/documents/2023-10/epa_sld_3.0_technicaldocumentationuserguide_may2021_0.pdf

### Social Vulnerability Index
https://www.atsdr.cdc.gov/placeandhealth/svi/documentation/SVI_documentation_2022.html

### USDA Food Access Research Atlas
https://www.ers.usda.gov/data-products/food-access-research-atlas/download-the-data/

In [1]:
import pandas as pd
from geopy import distance
import requests
import json
import datetime
import math
import itertools
import time
# Trespass Towing Report
tows = pd.read_csv('Trespass_Towing_Report.csv',
                   parse_dates=['Incident Date', 'Tow Date'])
# Proprietary Complaints Data
complaints = pd.read_csv('FY24 Trespass Tow Complaints(FY24 Trespass Tow Cases).csv',
                         parse_dates=['Create Date', 'Assigned Date', 'Closed Date'],
                         encoding='latin-1')


# Suppress scientific notation (for the GEOID)
pd.options.display.float_format = '{:.0f}'.format

## Cleaning Tow Reasons

In [3]:
# No parking (catch-all?)
spell_PKG = ['Unauthorized Parking', 'PKG', 'PGK', 'PKGG', 'NO PKG', 'NO PARKING', 'PKNG', 'NO PKNG', 'PARKED IN GRASS', 'PARIKING VIO', 'PARKING VIOLATION',
             'NO PKG ZONE', 'PRK', 'PARKED IN NO PARKING ZONE', 'PARKED ON THE GRASS', 'NO PARKING ZONE', 'UNAUTH PKG',
             'UNAUTHORIZED PARKING', 'NO PK ZONE', 'NPKG', 'PK', 'PKG`', 'PARKED OVER 2 HR LIMIT', 'ILLEGALLY PARKED ON GRASS',
             'NOT PARKED IN MARKED SPACE', 'NOT IN PARK SPACE', 'NOT PAKED IN MAKRED SPACE']
# Handicap space
spell_HANDI = ['Handicap Spot', 'HANDICAP', 'HANICAP SPACE', 'HANDICAPPED VIOLATION', 'HANDICAP PKG', 'HANDICAP PARKING', 'HAND', 'HANDY' 'IN HANDICAP',
               'PARKED IN HANDICAP', 'HADNICAP', 'HANDICAP VIOLATION', 'HADNICAPP', 'HANDI', 'PARKED IN HANDICAP WITH INVALID HANDICAP PERMIT']
# Fire lane
spell_FIRE = ['Blocking Fire Lane', 'FIRE', 'FIRELANE PARKING', "FIRE LANE", "FIRELAND NO PARKING ZONE", 'FIER', 'FIR', 'FIRE LANE AND BLOCKING ACCESS',
              'FIRE LN', 'FR', 'FIRELANE', 'FIRE LA', 'FIRE LANE VIOLATION']
# No overnight parking
spell_NOP = ['No Overnight Parking', 'NOP', 'NO', 'NO OVERNIGHT PARKING', 'NO[', 'VISITOR VIOLATION / NOP', 'NOPP', 'NO WALK OFF/ NO OVERNIGHT PARKING',
             'NO OVERNIGHT PKH', 'OVERNIGHT', 'NPO', 'NOPN', 'NO OVER NIGHT PARKING', 'UNAUTHORIZED PARKING - NO OVERNIGHT PARKING',
             'NOP`', 'RESERVED PKG NOP', 'MOP', 'NO OVERNIGHT PKG']
# Private Property
spell_PVT = ['Private Property', 'PVT', 'PRV PROP', 'PRI', 'PRIV PROP', 'PRVV', 'PRV PREOP', 'PRT PROP', 'PRIVATE PROPERTY', 'PRVT', 'PRV PROPERTY', 'PRV',
             'PRIVATE PROP' 'PRIVATE', 'PRV PROP', 'PRIV  PROP', 'PRV PROERTY', 'PRIV PROPERTY', 'PVI', 'PR', 'PRIVATE PROP	', 'PRIV',
             'PRIVATE PROP	']
# Abandoned Vehicle
spell_ABAN = ['Abandoned Vehicle', 'ABN', 'ABAN', 'ABAN AUTO', 'ABAND AUTO', 'ABND AUTO', 'ABN AUTO', 'ABD', 'ABANDON', 'ABANDONED', 'ABAND', 'ABANDON AUTO', 
              'ABANDONED AUTO', 'WALK', 'WALK OFF', 'WALKOFF', 'WALK-OFF']
# Repossesion
spell_REPO = ['Repossesion', 'REPO', 'REP', 'REOP', 'IMPOUND', 'IMP', 'REPOSSESSION']
# Expired Tags or Permit
spell_EXP = ['Expired Permit, Tags, or Registration', 'EXP', 'EXP PERMIT', 'EXP TAGS', 'EXP PERMIT', 'EXP  PERMIT', 'EXP TAG', 'EXP PASS', 'EXP REGISTRATION']
# Disabled Vehicle
spell_DAV = ['Disabled Vehicle', 'DAV', 'INOPERABLE', 'IN-OPERABLE', 'IMOBLE', 'IN OPERABLE VEH ', 'IN OPERABLE VEH', 'IMMOBILE CAR']
# Management request
spell_MNG = ['Management request', 'MNG', 'MANAGEMENT REQUEST', 'MGMT', 'MANAGEMENT REQ', 'UNAUTHORIZED PKG BY MANAGEMENT', 'MANAGER REQUEST', 'MGR REQ']
# Stolen
spell_ST = ['Stolen', 'STOLEN']
# Fake permit
spell_FAKE = ['Fake permit', 'FAKE']
# Unpaid
spell_UNPAID = ['Parking Unpaid', 'UNPAID', 'UNPAID PKG', 'NOT PAYING ON THE PARKING APP', 'NOT PAYING TO PARK']
# No permit
spell_NOPERMIT = ['No Permit', 'NO PERMIT']
#Invalid permit
spell_INVPERMIT = ['Invalid Permit', 'INVALID PERMIT', 'INV PERMIT', 'INVALID PERMIT DISPLAYED', 'ALTERED PERMIT', 'FAKE PARKING PERMIT', 'INVALID PERMIT COUPLE YEARS OLD',
                   'INVALID DISPLAY OF PERMIT']
# Visitor parking issues
spell_VISIT = ['Visitor Parking Violation', 'VISIT', 'VISITOR', 'EXP GUEST PASS', 'EXP VISITOR PERMIT', 'NO VIST PERMIT', 'INVALID PERMIT - VISITOR IN RESD SPOT ', 
               'INVALID PERMIT - VISITOR IN RESD SPOT', 'NO VIST PERMIT', 'VISITOR PARKED IN RESIDENT SPOT', 'RESIDENT PARKED IN VISITOR SPOT', 
               'INVALID PERMIT - VISITOR IN RESD SPOT ', 'EXP VISITOR PASS', 'EXP GUEST PASS', 'NO VISITOR PERMIT', 'EXP VISITOR PERMIT',
               'VISTOR PARKED IN RESD PARKING', 'VISITOR PARKED IN RESIDENT SPOT']
# Double parking
spell_DBL = ['Double Parking', 'DBL', 'DOUBLE PARKED', 'DBL PARKED', 'PARKED IN 2 SPACES', 'PARKED IN TWO SPACES', 'VEHICLE PARKED IN MULTIPLE SPACES',
             'TAKING UP TWO PARKING SPACES', 'TAKING 2 SPACES', 'TAKING UP 2 SPACES', 'TAKING UP TWO SPACES']

# Tow reason abreviations
spell_REASON = [spell_PKG, spell_FIRE, spell_NOP, spell_PVT, spell_ABAN, spell_REPO, spell_HANDI, spell_EXP, spell_DAV, 
                spell_MNG, spell_ST, spell_FAKE, spell_UNPAID, spell_NOPERMIT, spell_INVPERMIT, spell_DBL, spell_VISIT]

In [4]:
# Non-towing reasons in tow reasons column
not_reasons = ['HENRYS', '72', 'N', 'P', 'TCL', 'DIVERSIFY', 'DIVERSIFIED', 'TOW', '72HR', 'AUTH', 'HENRY', "HENRY'S TOWING", 'POL',
               'BIG TOW', 'MISC', 'VOP', 'PARKING', 'INV', 'COUNTY IMPOUND']
# Goes through 'Tow Reason' and 'Notes' columns to parse out true tow reasons
def find_reason(spell_R):
    # When the 'Tow Reason' is NaN
    if pd.isnull(row['Tow Reason']):
        # If notes is empty
        if pd.isnull(row['Notes']):
            tows.loc[index, 'Reason for tow'] = "Unknown"
        # If notes include reason
        elif any(spell in row['Notes'].upper() for spell in spell_R):
            tows.loc[index, 'Reason for tow'] = spell_R[0] # First spelling in each list is correct
    # If reason is spelled alternatively
    elif row['Tow Reason'].upper() in spell_R:
        tows.loc[index, 'Reason for tow'] = spell_R[0]
    # If reason is not a reason (error inputting variables)
    elif row['Tow Reason'].upper() in not_reasons:
        if pd.isnull(row['Notes']):
            tows.loc[index, 'Reason for tow'] = "Unknown"
        elif any(spell in row['Notes'].upper() for spell in spell_R):
            tows.loc[index, 'Reason for tow'] = spell_R[0]

# Loop through all the spellings
for spell in spell_REASON:
    for index, row in tows.iterrows():
        find_reason(spell)

In [5]:
# Filter out any reposessions
tows = tows[tows['Reason for tow'] != 'Repossesion']

## Cleaning Up Trade Names

In [6]:
# A1 Parking Management
spell_A1 = ['A1 Parking Management', 'A1', 'A1 PARKING', 'A1 PGK MNG', 'A1 PKG MGT', 'A1 PKG MANAGEMENT', 'A1 PARKING MANAGEMENT', 'A1 PKG MNG',
            'A1 PARKING MGMT', 'A1 PKG MGMT', 'A1 PKG', 'A1 PARKIGN', 'A-1 PARKING MNGT', 'A-1', 'A - 1', 'A ONE', '3014773521',
            'A ONE PARKING', 'A1-TOW', 'A1 TOWING', 'A ONE TOWING', '301 477 3521', 'AONE', 'AI Parking Management', '81 PARKING MANAGEMENT',
            'A1 PKG MANAGMENT', 'A1 PARK MGT', 'A1 PARK MGMT', 'A1 PRIVATE INVEST', 'A1 PROPERTY MGMT', 'AI PARKING MANAGEMENT', '301-477-3521', 
            'AI PARKING MGMT', 'AL PARKING MGMT', '(301) 477-3521']
# Aaron Murray
spell_AAV = ['Aaron Murray', '301-982-0091']
# A & G Auto Service
spell_AG = ['A & G Auto Service', 'A & G AUTO SERVICE', 'AMG RECOVERY']
# Airpark Autobody
spell_AIR = ['Airpark Autobody', 'AIRPARK', 'AIRPARK TOWING', 'AIR PARK', 'AIRPAIR TOWING', 'AIR PARK TOW', '301-948-7997', '948-7997', '301 948 7997', 
             'ARIPARK', '3019487997']
# AKJ Towing
spell_AKJ = ['AKJ Towing','AKJ', '301-774-0595', '301 774 0595', 'AKJ TOWING']
# All Towing Service LLC
spell_ALL = ['All Towing Service LLC', '240-528-1171']
# A & M Auto Service Inc.
spell_AM = ['A & M Auto Service Inc.', 'A & M AUTO SERVICE INC', 'AM TOWING', 'A&M', 'A AND M', '3014210953', '301-421-0953', '301.421.0953', '301 421 0953',
            'A&M TOWING', 'A&M 301 421 0953', 'A&N TOWING', 'A%M']
#Auburn Towing, Inc.
spell_AUB = ['Auburn Towing, Inc.', 'AUBURN', 'AUBURN TOWING', 'AUBURN TOW', 'AUBBURN', 'AUBRUN', 'AUBRURN', 'AIUBURN', 'A UBURN', 'AUBRUN TOWIN',
             'AUBUBRN', 'AUBRUN TOWING', 'AURBURN', 'AUBURB', 'AUBURNTOWING', 'AUBURBN', '301-977-2740', '301 977 2740', '3019772740', 'AUBERN', 
             '301-9772740', 'AUBUIRN']
# Automotive Support Services
spell_AUTO = ['Automotive Support Services', '301-384-4869']
#Big Tow
spell_BIG = ['Big Tow', 'BIG TOW', 'BIG', 'BIG TOW TOWING', 'BIGTOW', 'BIG TOWING', 'BIG BIG TOW', 'BIG TOE', 'BG TOW', 'BIGTOW TOWING', 'BIG TOWN', 
             'BGTOW', 'BOG TOW', 'BIG TOWI', '424-4869', 'BIG TO', 'BIIG TOW', 'BOGTOW']
#Diversified Recovery
spell_DIVERS = ['Diversified Recovery', 'DIVERSIFIED', 'DIVERSIFED', 'DIVERSIFIED TOWING', 'DIVERSFIED', 'DIVERSITY TOWING', 'DIVERSIFY RECOVERY', 'DIVERSIFIED 301-881-8035',
                'DIVERSIFY', 'DIVERSIFIED TOWING AND RECOVERY', 'DIVERSFD', 'DIVERISFIED', 'DIVERSIFIED RECOVERY', 'DIVERSIIED',
                'DIVERSIFIED TOW', '301-881-8035', 'DIVIERSIFIED TOW', 'DIVERSFY', 'DIVIERSIFIED', 'SUBVERSIFIED', 'DIVERSIDIED', 'DIVERISIFIED', 
                'DIVER', 'DIVERS', 'DIVERS', 'DIVERSERIFED', 'DEVERSIFIED', 'DIVERDIFIED', 'DIVERIFISED', 'DIVERSERIFED TOWING', 'DIVERSIFED TOWING',
                'DIVERSIFIFED', 'DIVIRSIFIED TOW', 'DOVERSIFIED', '301 881 8035', 'DIVSFY']
# E & S Towing Service
spell_ES = ['E & S Towing Service', 'E& S TOWING', 'E&S', 'E & S', 'E AND S TOWING', 'E & S TOWING', 'ES TOWING', 'E&S TOWING', 'E/S', 'E AND S',
            '240 304 9367', '2403049367', '240-304-9367', '240-304-9467', ' E S ', '240-304-1643', '240 304 1643']
# Fletcher's Service Center
spell_FLETCH = ["Fletcher's Service Center", '301 924 4500', 'FLETCHERS TOWING']
# Grace Towing and Recovery, LLC
spell_GRACE = ['Grace Towing and Recovery, LLC', 'GRACE TOWING', 'GRACE TOW AND RECOVERY', 'GRACE TOW', 'GRACE TOWING AND RECOVERY', 'GRACE', 'GRACE TOWING & RECOVERY',
               'GRACE RECOVERY', 'GRACES', '240 477 7191', '240-477-7191', 'GREYS', 'GRAYS', 'GRAYS TOWING AND RECOVERY', 'GRATES', 'GRAYS TOWING']
# Henry's Wrecker Service
spell_HENRY = ["Henry's Wrecker Service", 'HENRYS', 'HENRYS TOW', "HENRY'S", "HENRY'S TOWING", "HERNYS", "HENRYS'", 'HENRYS`', "HENRY'S TOWINH"
               'HENRY', "HENRY'S TOWINH", 'HENRYS TOWING', 'HEN','HNERYS', 'HENYRS', "HENYR'S", "HENRYS'", '3019270007',
               'HENRY"S', '301 869 4800', "HERNY'S TOWING", 'HENRY', 'HERNSY', "HNERY'S", '301-869-4800', 'HENERYS TOWING', "HENRY'S`", 'HE NRYS',
               'HNERYS TOWING', 'HENREYS', 'HEBRYS', 'FHENRYS', 'HENRY;S', 'HENRY TOWING', "HENRY'SC", 'HNERYS TOWING', '3018694800',
               'HENERYS', "HENRY'S LOT", 'HENRS', 'HENRYD', 'HENRYS TOWNING', 'HENRYS TWOING', 'HENRYSS', "HERNY'S", "HENRY'S TOW", 'HENRYS TOWINF',
               'HERNY', '301-927-0007', '927-0007', 'HERYS', '301 927 0007', '(301)927-0007', '(301) 927-0007', 'HRNYS', '(301) 869-4800', 'HESNRYS TOWING']
# iTow Towing
spell_ITOW = ['iTow Towing', '240 498 2408']
# K & D Towing
spell_KD = ['K&D', 'K & D', '301-399-9225', 'K AND D', 'K AND D TOWING', 'K&D TOW', 'K&D TOWING', 'K N D TOWING', 'KD TOWING', 'KND', 'K&B',
            'KND TOWING''K & D TOWING', 'KND TOWING', '301 755 4450', 'K & D TOWING', 'KD', 'A&D', 'K AND B', '3017554450', 'KANDD', 'KAND TOWING',
            'A&D TOWING', 'K &D', 'K & D  TOWING', 'K& D', '301 755 1450', '301-755-4450', 'KNE TOW', 'K&E', 'KAND D', 'KK&D', 'AD TOWING', 'K&B TOWING']
# L.L. Towing
spell_LL = ['L.L. Towing', 'LL TOWING', '(301) 774-3342', '301 774 3342', '301-774-3342']
# Richard Lofton
spell_LOFT = ["Richard Lofton", 'LOFTONS', 'LOFTONS TOW', 'LOFTONS TOWING', 'LOFTON TOWING', "LOFTON'S", 'LOFTON', 'LOFTON TOW',
              '240-388-1389', 'LAWFTONS', 'LAWSON TOWING', 'LAWSONS', '2403881389']
# MEGEX Towing and Recovery
spell_MEG = ['MEGEX Towing and Recovery', '202-751-8300', 'MEGEX']
#Metro Investigation & Recovery Solutions
spell_METRO = ['Metro Investigation & Recovery Solutions', 'METRO INV', 'METRO INVEST', 'METRO INVESTIGATIONS', 'METRO INVESTIGATONS', 'MET INV', 'MET INVEST', 'METRO IN', 'METRO INC',
               'METRO', 'METRO INVESTIGATION', 'METRO I NV', 'METRO INVSETIGATIONS', 'METRO INVESTIGATIONS TOWING', 'METRO INV & REC',
               'METOR INVESTIGATION', 'METRO INVESTIGAQTIONS', 'METRO INVESTIGAT',  'METRO INV TOW', 'MERTO INV', 'METRO INV & RECOVERY',
               'METRO  INV', 'METRO INVESTGIATION', 'METRO INVSTIGATIONS', 'METRO INVES','METRO INVESTIIGATIONS', 'METRO INV RECOV', 'METRO INV.',
               'METRO INVESTIGATIONS TOW', 'METRO INVESTGATIONS', 'METRO INVEST TOW', 'MERO INV', 'METOR', 'METERO', '1101 GALASKY HWY',
               'METRO INVEST.', 'METRO INVESTIAGTION', 'METRO INVESTIG', 'METRO INVESTIGATION AND RECOVERY', 'METRO INVESTIGATION RECOV', 'METRO INVESTIGATIOS',
               'METRO INVS', 'METRO INVSETIGATION', 'METRO INVST', 'METRO INVSTEGATIONS', 'METRO TOWING', 'METROINVESTIGATIONS', 'METRO INVESTAGATION',
               'FINAL NOTICE', 'FINAL NOTICE TOW LOT', 'FINALNOTICE', '410-344-1501', 'METO INV']
# Millers's Towing & Recovery, LLC
spell_MILLER = ["Millers's Towing & Recovery, LLC", "MILLER'S TOWING & RECOVERY, LLC", 'MILLERS', 'MILELRS', "MILLER'S TOWING", 'MILLER S', 'MILLERS TOWING', 'MILLER', "MILLER'S", 'MILLERSS',
                'MILLERS TOW', ',ILLERS', 'MILLER TOWING', "MILER'S", 'MILILERS', 'MILLESR', 'MILLERSQ', "MILLER'S TOWING AND RECOVERY", 'MILLERS TOWING AND RECOVERY',
                'MILLERS TOWING & RECOVERY', 'MILLERS & RECOVERY', 'MILLER WOTING', 'MILL.ERS', 'MILLERS RECOVERY', '301-602-0392', 'MILLER TOWING AND RECOVERY', '301 602 0392', 
                'MILLER TOW', "ILLER'S TOWING", '301.602.0392', 'MEYERS']
# Montrose Towing
spell_MON = ['Montrose Towing', 'MONTROSE TOWING', 'MONTROSE']
# Morton's Towing & Recovery, Inc.
spell_MORT = ["Morton's Towing & Recovery, Inc.", 'MORTONS TOWING', 'MORTONS', "MORTON'S", 'MOTENS', ' 301-330-1170', "MORTIN'S", 'MORTON', 'MORTANS', 
              '3013301170']
# N & S Towing
spell_NS = ['N & S Towing', 'NS TOWING', 'N & S TOWING', 'N&S TOWING', 'N&S', '301-595-3338']
# Ricky's Towing & Recovery, LLC
spell_RICK = ["Ricky's Towing & Recovery, LLC", "RICKY'S", 'RICKYS', 'RICKYS TOWING', "RICKEY'S", 'RICKEYS', 'RICKY', "RICKY'S TOWING", 'RICKYS TOW', 
              'RICKSY', 'RICKSYS', 'RICKY TOW', 'RICKIES', "RICKY'S  TOWING", "RICK'YS", 'RICKEY', 'RICK', 'RICKYSTOWING', 'RICYS TOWING', '240-461-9995', 
              'RICYS']
# Scott Simpson Towing
spell_SCOTT = ['Scott Simpson Towing', 'SCOTT SIMPSON TOWING', 'SCOTTS SIMPSON TOWING', 'SCOTT SIMPSON', '301 476 9090', 'SST TOWING']
# Silver Spring Towing and Recovery
spell_SILV = ['Silver Spring Towing and Recovery', 'SILVER SPRING TOWING', '301-589-7878', 'SS TOWING', '301 589-7878', '(301)589-7878', '301 589 7878',
              'SILVER SPRING TOW', '2406876994', '3015897878']
# Speedy Towing
spell_SPEEDY = ['Speedy Towing', 'SPEEDY', 'SPEEDYS', 'SPEEDY TOWING', 'SPEEDY TOW', 'SPEEYING', 'SPEEEDY', 'SPEEYD', 'SPEEDY TOWIG',
                'SPEEDY TOWNG', 'SPEEDYS TOWING', 'SPEEDING TOWING', 'SPEDDY', 'S[PEEDY', '301-588-1322', 'SPEDDY TOWING', 'SPEDDY',
                'SPEDY', 'SPOEDY', 'SPEEDSY', 'SPEEDYS`', 'SPEEDSY', "SPEEDY'S", 'SILVER SPRING', 'SPEEDY LOT', '3015881322', 'SPEEDI',
                'BD TOWNG', '301 588 1322', 'SPEEDYDS', 'SPEED Y', 'SPEED', 'SPEEYDS']
#Stronghold Towing
spell_STR = ['Stronghold Towing', '301-916-0660', 'STRONGHOLD']
#Werking Automotive
spell_WERK = ['Werking Automotive', 'WERKING', '301-916-0455', '3019160455', 'WERKING TOWING', '301 916 0455']

# Ambiguous/unsure trade names
# All Towing Service LLC
spell_ALL = ['All Towing Service LLC', '240-528-1171', 'RD TOWING', 'RND', 'R&D TOWING', '240-528-1171']
# AMJ Recovery Service?
spell_AMJ = ['AMJ Recovery Service?', 'AMJ', 'AMJ RECOVERY', 'AMJ RECOV', 'AMJ TOWING']
# Apex Towing and Recovery
spell_APEX = ['Apex Towing and Recovery', 'APEX']
# Asikin Towing?
spell_ASI = ['Asikin Towing', '301-530-2888', 'ASIKIN TOWING', 'ASKIN', 'ASIKIN', 'ASKIN TOWING']
# AT Auto Transort?
spell_AT = ['AT Auto Transport', '240-529-9839', '240 529 9839']
# Authorized?
spell_AUTH = ['Authorized Towing', 'AUTHORIZED', 'AUHTORIZED', 'AUTH', 'AUTHORIZED TOWING', 'AUTHORIIZED', 'AUTHOZIED TOWING '
              'AUTHORIZE TOW', 'AUTHORIZE', 'AUTHORIZE TOE', 'AUTHORZIED', 'AUTHROIZED','AUTHORIZED TOWNG',
              'AUHTORIZED TOWING', 'AUTORIZED TOWING', 'AUTHORIZE TOWING', 'AUTHRORIZED', 'AUTH', 'AUJTHORIZED',
              'AUTHTORIZED', 'AUTHORIRZED', 'AUTHORZED', '301-562-4020', 'AUTORIZED', 'AUTHORIZIED', 'ATUHORIZED',
              'AUTHROZIED', 'AUTHORIZED TOWIN', 'AUTHORUIZED', 'AUTHORIZED`', 'AUTHROZIED TOWING', 'AUTH`', '301-562-4020',
              'AUTHORIZE TOW', '16327', 'ATHORIZED', 'AUTHORIZE TOW', 'AUTHORIZZED', 'AUTHORUZED', 'AUTHOTIZED',
              'AUTHPRUIZED', 'AURHORIZED', 'AUTHOIRZED', 'AUTHOIZED', 'AUTHORIE TOWING', 'AUTHORIZE TOW', 'AUTHORIZEED',
              'AUTHOZIED', 'AUTHOZIED TOWING', 'AUTHPRIZED', 'AUTOHERIZED', 'AUTHORIZED TOW', '31-562-4020', '301.562.4020',
              '301 562 4020', 'AUT', 'ZUTHORIZED', 'AIUTHORIZE', 'UTHORIZED', 'AIUTH', '301 5624020', '3015624020']
# Balto Auto Sales?
spell_BALT = ['Balto Auto Sales', '443-770-0320']
# Benitez Towing, LLC
spell_BENI = ['Benitez Towing, LLC', 'BENITEZ']
# Bill Hallet's Towing?
spell_BILL = ["Bill Hallet's Towing", 'HALLET TOWING', '301-424-1010']
# Cherner?
spell_CHER = ['Cherner', 'CHERNER BROTHERS', 'CHERNER BROTHER', 'CHERNER', 'CHERNER BROS', '301-949-9095', 'CHERNER BROTHERS`']
# Cody Towing?
spell_CODY = ['Cody Towing', 'CODY', 'CODY TOWING AND RECOVERY', 'CODYS', '3014734322']
# CTI?
spell_CTI = ['CTI Towing', 'CTI', 'CTI TOWING', '301-772-2998', '301 772 2998', 'CIT', 'CTI RECOVERY']
# Custom Towing & Auto Repair
spell_CUST = ['Custom Towing & Auto Repair', '301-253-5151', 'CUSTOM TOWING', '301 253 5151', 'CUSTOM', 'CUSTOMS', 'CUSTOMS TOWING']
# Drive?
spell_DRIVE = ['Drive Towing', 'DRIVE TOW', 'DRIVE TOWING', '(301) 585-8808', '301-585-8808', '301 585 8808', '(301)585-8808',
               '301585 8808', 'DRIVE', 'DRIVE TOW LOT', 'DRIVE`', 'DRIVW', 'DRY TOWING', 'DIVE', 'DRVE', '3015858808', 'FRIVE', 
               'STRIVE']
# EC?
spell_EC = ['EC', 'EC TOWING', 'EC TOW', '202 603 9543', '240-382-2621', '240 382 2621', '202-603-9543', 'EASY', 'EC 301 603 9503'
            'EASY TOWING', 'E&C', 'E&C TOWING', 'E-C TOWING', 'E Z TOWING', 'EC', 'EZ TOWING', 'EASY TOWING', 'E C TOWING', 'EC 301 603 9503',
            'E.C.', 'EC TOWNG','EZ', '240 382 2048', '240-382-2048', '(240) 382-2621', '240-382-2661', 'DC TOWING']
# ETC Recovery?
spell_ETC = ['ETC Recovery', 'ETC', 'ETC RECOV', 'ETC RECOVERY', 'ETC TOWING']
# Fast Guys Towing & Roadside Assistance?
spell_FAST = ['Fast Guys Towing & Roadside Assistance', 'FAST GUYS', 'FAST GUY', 'FAST', 'FAST GUYS TOWING', 'FASTCAR']
# GTS Towing?
spell_GTS = ['GTS Towing', '703-462-0447']
# Greenwood Recovery?
spell_GREEN = ['Greenwood Recovery', 'GREENWOOD RECOVERY', 'GREENWOOD', 'GREENWOOD RECOV', 'GREENWOOWD REC', 'GREEWOOD', 'GRNWOOD', 
               'GRNWOOD RECOV', '410-383-8000', '410 383 8000']
# JH Recovery LLC?
spell_JH = ['JH Recovery LLC', 'JH', 'JH RECOV', 'JH RECOVERY', 'JH TOWING']
# JNJ Towing & Recovery?
spell_JNJ = ['JNJ Towing & Recovery', '240-708-5399']
# Tow Kingz?
spell_KING = ['Tow Kingz', '5404268799', '540-426-8799', 'TOWKINGZ']
# LED?
spell_LED = ['LED', 'LED TOW', 'LED TOWING', 'LED RECOVERY', 'LED', '301-556-8006', 'LED TOWIN', '301 556 8006']
# L & K?
spell_LK = ['L&K', 'L&K RECOVERY', 'L&K TOW', 'L AND K TOWING', 'L&K RECOVERY', 'LK RECOVERY', 'LMK RECOVERY', 'LK COMPANY',
            'LNK RECOVERY', 'L AND K', 'L & K RECOVERY', 'L&K', 'L AND K RECOVERY', 'LNK', '703-327-5600', 'L & K', 'L& K', 'L&K RECOV', 
            'LNK', 'L&K ROCVOERY', 'L&K TOWING', 'L&L RECOVERY', 'LNT TOWING', 'L AND K']
# Late Nite Towing?
spell_LN = ['Late Nite Towing', '240-602-4944', 'LATE NIGHT TOWING', 'LATE NITE TOWING', 'LATE NIGHT']
# LTS Repossession?
spell_LTS = ['LTS Repossession', 'LTS', 'LTS RECOVERY']
# Milestone Towing & Transport?
spell_MILE = ['Milestone Towing & Transport', 'MILESTONE', '301-330-1614']
# Murphy's Towing
spell_MURP = ["Murphy's Towing", '202-359-9585']
# GTS Towing?
spell_GTS = ['GTS Towing', '703-462-0447']
# On the Run Towing?
spell_OTR = ['On the Run Towing', 'ON THE RUN', 'ON THE RUN TOW', 'ON THE RUN TOWING', 'ONTHERUN', '301-740-4600', 'ONE THE RUN', 
             '3017404600']
# Prestige?
spell_PRES = ['Prestige', 'PRESTIGE', 'PRESTIGE TOW', 'PRESIGE', 'PRESTIGE TOWING', 'PRESITGE', 'PRES', 'PRESTIGE'
              'PRESTAGE', 'PESTIGE', 'PRETSIGE', 'PRESTIEGE', 'PRESTAGE', 'PRESTIUGE', 'PRESTGE', 'PRESTEIGE', 'PRESTIVE',
              'PRESTIGE AUTO', 'PREISTIGE', 'PRSTIGE', '[RESTIGE', 'PRETIGE', 'P RESTIGE', 'PRISTIGE', '301-377-9285',
              'PRESTIAGE', 'RPESTIGE', 'PRESTIGE`', 'PREST IGE', 'PRESTIGO', 'PRESITGE TOWIN', 'PRESTIG', '3013779285', 
              'PRESTOGE', 'PRESITIGE', 'PRESTIGG', 'PREST', 'PRESTTIGE', 'PRESITGE TOWING', '301 377 9285', 'PR;ESTIGE',
              'PRETIDGE']
# RBF Towing?
spell_RBF = ['RBF Towing', '(240) 793-2272', '240 793 2272', 'RBF', 'RBF TOWING', '240-793-2272', 'RDF TOWING', 'RVF TOWING',
             'RVF', 'RBS TOWING']
# Roadmate Towing?
spell_ROAD = ['Roadmate Towing', '240-641-2188']
# Thomas Recovery Inc?
spell_THOM = ['Thomas Recovery Inc', '410-247-0662', 'THOMAS RECOVERY', '410 247 0662', 'THOMAS', 'THOMASRECOVERY']
# Tri State Recovery?
spell_TRI = ['Tri State Recovery', 'TRI STATE RECOVERY']
# TRP?
spell_TRP = ['TRP', 'TRP', 'TRP TOWING', '301-214-2490', '301 214 2490', 'PRP', 'PRP TOWING', 'TRP TOW', 'TRP RECOVERY', 
             'THOMAS ROBERT PAUL TOWING', '3012142490', 'T I P']

# All spelling list names
spelling = [spell_A1, spell_AG, spell_ALL, spell_AIR, spell_AKJ, spell_AUB, spell_AUTH, spell_MILE,
            spell_AUTO, spell_BIG, spell_DIVERS, spell_DRIVE, spell_EC, spell_ES, spell_ASI, spell_ROAD,
            spell_GRACE, spell_HENRY, spell_KD, spell_METRO, spell_MILLER, spell_MORT, spell_BILL,
            spell_PRES, spell_RICK, spell_SILV, spell_SPEEDY, spell_LK, spell_LED, spell_TRP, spell_RBF,
            spell_LOFT, spell_MON, spell_GREEN, spell_AMJ, spell_CODY, spell_CUST, spell_OTR, spell_WERK,
            spell_AM, spell_LN, spell_SCOTT, spell_GREEN, spell_NS, spell_CTI, spell_LL,
            spell_STR, spell_AT, spell_FAST, spell_JNJ, spell_CHER, spell_THOM, spell_BALT, spell_GTS,
            spell_ETC, spell_JH, spell_LTS, spell_FLETCH, spell_ALL, spell_AAV, spell_ITOW, spell_MEG, 
            spell_APEX, spell_TRI, spell_BENI, spell_KING, spell_MURP]


In [7]:
# Parses out merchant from 'Storage Company' and 'Notes' columns
def find_trade_name(spell_COMP):
    spell_alt = ['TCL', 'COUNTY LOT', 'TC;', 'TCK', 'TC:', 'COUNTY', 'VRS', "UNKNOWN",
                 'PKG', 'FIRE', 'NOP', 'PVT', 'ABAN', 'ABN', 'REPO', 'AAU', 'ABAN AUTO', 'ALLY', 'CAPITAL ONE',
                 'CAPITAL', 'CARMAX', 'CHASE', 'ENTERPRISE', 'ECC', 'HERTZ', 'PNC BANK', 'ROYAL TRUST',
                 'SANTANDER', 'STATE EMPL CREDIT', 'TCCL', 'TC:', 'TCKL', 'ONE MAIN', 'FLAGSHIP CREDIT',
                 'FLAGSHIP', 'ANDREWS FCU', 'ALLY', 'ALLY FINANCIAL', 'FIRST HELP FINANCIAL', 'OTHER',
                 'CREDIT ACCEPTANCE', 'AMERICAN HONDA FINANCE', 'AMERICAN HONDA', 'ANNAPOLIS JUNCTION', 
                 'AVS', 'BRIDGECREST', 'BRIDGECREST CREDIT', 'CREDIT ACCEPTANCE', 'CONSUMER', 'CREDIT UNION LLC',
                 'CT HOLDINGS', 'EXETER', 'EXETER FINANCE', 'EXP PERMIT', 'GLENMONT FINANCIAL', 'FRIENDLY',
                 'GM FIANCE', 'P&CC BANK', 'WELLS FARGO FINANCIAL', 'ALLIED FINANCIAL', 'ALLIED SOLUTIONS',
                 'AMERICAN', 'AMERICA HONDA', 'AMERICAN CREDIT ACCEPTANCE', 'AVAS RENTAL CAR', 'AVIS', 'AVIS LOT',
                 'AVIS RENTAL LOT', 'CHRS CAPITAL', 'FIANCE CO C/O TCL', 'FINANCE TO VIA TCL', 'KENSINGTON FINANCE', 
                 '1ST CORPORATION', '305 METROPOLITAN GROVE RD', '305 METROPOLITAN GROVE RD', '446 N FREDERICK ATV',
                 'HERTZ RENTAL', 'LOT', 'PRV', 'SANTANDER FINANCE', 'SS TCL', 'TC', 'WESTLAKE', 'TCLE', 'AAV',
                 '305 METROPOLITIAN GROVE RD', 'AM VRS', 'T CL', 'EXP TAGS', 'TCL;', 'TCL+', 'TL', 'TCLX',
                 'TCL 3013779285', 'TOYT', 'TCL- FINANCE CO', 'ALLIED', 'ALLIED BANK', 'AMERICAN CREDIT',
                 'AMERICAN CREIT', 'AMERICAN HOND', 'AUTO TRACK', 'AVAS RENTAL CAR', 'AVIS CAR RENTAL',
                 'AVIS LOT', 'AVIS RENTAL LOT', 'CHRY CAPITAL', 'COUNTY IMPOUND', 'CREDIT ACCEPTANCE', 'L'
                 'CREDITA FINANCE', 'CT HOLDINGS', 'HYUN CAPITAL', 'TCL`', 'TCL3018694800', 'TCLNE', 'TO ABN',
                 'TOYOTO FINANCIAL', 'TOYT FINANCIAL', 'WESTLAKE FINANCIAL', 'CBA FINANCE', 'CBAB FINANCE', 'CL',
                 'HERTZ RENT A CAR', 'HONDA FINACIAL', 'PNC', 'RIDGECREST', 'ES', 'EXITER', 'EXP', 'HON', 'INVALID PERMIT',
                 'LK', 'MERC BENZ', 'CTL', 'CAPITAL ONE BANK', 'CREDIT ACCEPTANCE', 'LANGLEY', 'LIENHOLDER LOT',
                 'LENDER LOT', 'LIENHOLDER LOT', 'AAV TOWING', 'ACCEPTANCE', 'FORUM CREDIT UNION', 'SECU', 'SPECTRA', 
                 'TC L', 'TCI', 'TLC', 'TOW', 'YVL', 'PAR', 'PAR SERVICES', 'PAR SVC', 'PAR SVCS', 'PAR SERVICES INC',
                 'PAR SERIVCES', 'TXL', 'TCLL', 'TPPD', 'LOT', 'TCK', 'IMPOUND', 'L', 'PK', 'TCP', 'TCLA', 'TCXL', 'TVL', 
                 'TTCL', 'TCL `', 'LOY', 'SECURITY TOWING', 'RCL', 'R&D', 'R&B']
    # When the 'Storage Company' is NaN
    if pd.isnull(row['Storage Company']):
        if pd.isnull(row['Notes']):
            tows.loc[index, 'Trade Name'] = "Unknown"
        elif any(spell in row['Notes'].upper() for spell in spell_COMP):
            tows.loc[index, 'Trade Name'] = spell_COMP[0]
    # When the 'Storage Company' is not a company
    elif row['Storage Company'].upper() in spell_alt:
        if pd.isnull(row['Notes']):
            tows.loc[index, 'Trade Name'] = "Unknown"     
        elif any(spell in row['Notes'].upper() for spell in spell_COMP):
            tows.loc[index, 'Trade Name'] = spell_COMP[0]
    # When the 'Storage Company' is one of the spelled companies
    elif row['Storage Company'].upper() in spell_COMP:
        tows.loc[index, 'Trade Name'] = spell_COMP[0]
    # When the 'Storage Company' is abandoned??
    elif row['Storage Company'].upper() in spell_ABAN:
        if pd.isnull(row['Notes']):
            tows.loc[index, 'Trade Name'] = "Unknown"
        elif any(spell in row['Notes'].upper() for spell in spell_COMP):
            tows.loc[index, 'Trade Name'] = spell_COMP[0]

# Loop through all the spellings
for spell in spelling:
    for index, row in tows.iterrows():
        find_trade_name(spell)

In [25]:
epa = pd.read_csv('EPA_SmartLocationDatabase.csv')
# Filter for relevant columns
epa = epa[['TRACTCE',
                             'D4D', #Aggregate frequency of transit service [D4c] per square mile
                             'D1A', # Gross residential density (HU/acre) on unprotected land
                             'D1B', # Gross population density (persons/acre) on unprotected land
                             'D1C', # Gross employment density(jobs/acre) on unprotected land
                             'D1D' # Gross activity density (employment + HUs) on unprotected land
                             ]] 
epa.columns = ['tract', 'transit_freq', 'resident_density', 'pop_density', 'employ_density', 'act_density']
# Convert to string for merging
epa['tract'] = epa['tract'].astype(str)
# Average block groups for tracts
epa = epa.groupby('tract').mean()

### Census Queries

In [11]:
import requests

HOST = 'https://api.census.gov/data'
year = '2022'
dataset = 'acs/acs5' # American Community Survey 5-Year Estimates
base_url = '/'.join([HOST, year, dataset])

get_vars = [
    'B19013_001E', # Median household income in the past 12 months (in 2023 inflation-adjusted dollars)
    'B05012_003E', # Estimate total foreign-born population
    'C16001_002E', # Estimate total speak only English at home
    'B01003_001E', # Estimate total population
    'B08006_002E', # Estimate total means of transportation to work - car, truck, or van
]
predicates = {}
predicates['get'] = ",".join(get_vars)
predicates['for'] = 'tract:*'
predicates['in'] = 'state:24;county:031'

r = requests.get(base_url, params=predicates)

# Column names for df
col_names = [
    'median_household_income',
    'foreign_born_population',
    'speak_only_english',
    'total_population',
    'means_of_transportation_to_work_car_truck_or_van',
    'state',
    'county',
    'tract'
]

# Covert r to pandas df
acs_data = pd.DataFrame(data=r.json()[1:], columns=col_names)

In [12]:
col_numeric = [
    'median_household_income',
    'foreign_born_population',
    'speak_only_english',
    'total_population',
    'means_of_transportation_to_work_car_truck_or_van']

for col in col_numeric:
    acs_data[col] = pd.to_numeric(acs_data[col], errors='coerce')

In [ ]:


acs_data = acs_data.groupby(['state', 'county', 'fips_no_block']).agg({
    'median_household_income': 'mean', 
    'foreign_born_population': 'sum',
    'speak_only_english': 'sum', 
    'total_population': 'sum',
    'means_of_transportation_to_work_car_truck_or_van': 'sum'
}).reset_index()

In [19]:
cei = pd.read_csv('cei.csv')
cei = cei[['TRACTCE20', 'cei', 'cat_cei']]
# Make tract string for purpose of merging
cei['TRACTCE20'] = cei['TRACTCE20'].astype(str)

In [17]:
tows['TractFIPS'] = tows['geoid'].astype(str).str.slice(5,11)

In [31]:
tows = tows.merge(acs_data, left_on='TractFIPS', right_on='tract', how='left', validate='m:1')

In [21]:
tows = tows.merge(cei, left_on='TractFIPS', right_on='TRACTCE20', how='left', validate='m:1')

In [28]:
tows = tows.merge(epa, left_on='TractFIPS', right_on='tract', how='left', validate='m:1')

In [33]:
tows_trimmed = tows.drop(['Unnamed: 0', 'Tow Reason', 'Incident Date', 
                  'Geo-location field', 'coord', 'Storage Company', 'TRACTCE20',
                  'state', 'county', 'tract'], axis=1)

In [37]:
tows_trimmed.to_csv('tows.csv', index=False)